In [2]:
import keras
print(keras.__version__)

fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

2.4.3


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin


class ImageFloatConverter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        from skimage import img_as_float 
        import numpy as np
    
        X_ = np.zeros(X.shape, dtype=np.float)
        for i in range(X.shape[0]):
            X_[i] = img_as_float(X[i])
        return X_
    
scaler = ImageFloatConverter()
x = scaler.transform(X_train)



In [6]:
import numpy as np

# Test scaler max deviation
np.max(np.abs(x[42] * 255 - X_train[42]))

2.842170943040401e-14

In [7]:
class ImageAsSerial(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        import numpy as np
        X_ = np.zeros((X.shape[0], X.shape[1] * X.shape[2]), dtype=np.float)
        for i in range(X.shape[0]):
            X_[i] = X[i].ravel()
        return X_

In [8]:
# Test that the Pipeline is working - not necessary to run.

from sklearn.linear_model import SGDClassifier

pipe = Pipeline([
    ('ImageFloatConverter', ImageFloatConverter()),
    ('Serialize', ImageAsSerial()),
    ('SGDClassifier', SGDClassifier(max_iter=100, tol=1e-3))])


pipe.fit(X_train, y_train)

Pipeline(steps=[('ImageFloatConverter', ImageFloatConverter()),
                ('Serialize', ImageAsSerial()),
                ('SGDClassifier', SGDClassifier(max_iter=100))])

In [9]:
pipe.score(X_test, y_test)

0.8327

# Test af modeller

In [10]:
from sklearn.linear_model import SGDClassifier

model = Pipeline([
                ('ImageFloatConverter', ImageFloatConverter()),
                ('Serialize', ImageAsSerial()),
                ('SGDClassifier', SGDClassifier(max_iter=100, tol=1e-3))]
            )
            
model.fit(X_train, y_train)
sgd_score = model.score(X_test, y_test)
print(sgd_score)

0.8294


In [11]:
from sklearn.svm import LinearSVC

model = Pipeline([
                ('ImageFloatConverter', ImageFloatConverter()),
                ('Serialize', ImageAsSerial()),
                ('LinearSVC', LinearSVC())]
            )
            
model.fit(X_train, y_train)
lsvc_score = model.score(X_test, y_test)
print(lsvc_score)

/home/morten/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
0.8396


In [12]:
from sklearn.neighbors import KNeighborsClassifier
model = Pipeline([
                ('ImageFloatConverter', ImageFloatConverter()),
                ('Serialize', ImageAsSerial()),
                ('KNeighborsClassifier', KNeighborsClassifier())]
            )

model.fit(X_train, y_train)
knn_score = model.score(X_test, y_test)
print(knn_score)

0.8554


In [13]:
from sklearn.neural_network import MLPClassifier

models = Pipeline([
                ('ImageFloatConverter', ImageFloatConverter()),
                ('Serialize', ImageAsSerial()),
                ('MLPClassifier', MLPClassifier())]
            )

model.fit(X_train, y_train)
mlp_score = model.score(X_test, y_test)
print(mlp_score)

0.8554


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

# SOURCE: https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-fashion-mnist-clothing-classification/

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)
model = define_model()
model.fit(trainX, trainY)
eval = model.evaluate(testX, testY)

313/313 [==============================] - 1s 4ms/step - loss: 0.3250 - accuracy: 0.8837


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               540900    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 542,230
Trainable params: 542,230
Non-trainable params: 0
_________________________________________________________________


In [17]:
scores = {} # Reset
attempt = 0

In [24]:
import keras
from keras import layers

model = keras.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(3,3),
                        activation='relu', input_shape=(28, 28, 1)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))
model.add(layers.Dense(units=84, activation='relu'))
model.add(layers.Dense(units=10, activation='softmax'))

#model.summary()

X_tmp = X_train.reshape((X_train.shape[0], 28, 28, 1))
y_tmp = to_categorical(y_train)
X_tmp_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
y_tmp_test = to_categorical(y_test)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.fit(X_tmp, y_tmp)
eval = model.evaluate(X_tmp_test, y_tmp_test)
scores[f'attempt{attempt}'] = eval
attempt = attempt + 1

313/313 [==============================] - 1s 3ms/step - loss: 0.4075 - accuracy: 0.8481


In [21]:
print(scores)

{'attempt0': [0.382768839597702, 0.864799976348877]}
60000


In [23]:
model = keras.Sequential()
model.add(Flatten(input_shape=(28,28,1)))
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(X_tmp, y_tmp)
eval = model.evaluate(X_tmp_test, y_tmp_test)
model.summary()

ValueError: in user code:

    /home/morten/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/morten/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/morten/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/morten/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/morten/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/morten/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/